In [2]:
from pathlib import Path
import collections
import pandas as pd

LOG_PATH = Path.cwd().parent / "data-cyber-journey-2026" / "data" / "auth_log_sample.txt"
LOG_PATH

PosixPath('/Users/baileylyback/Developer/github/data-cyber-journey-2026/data/auth_log_sample.txt')

In [3]:
rows = []

with LOG_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        if "Failed password" not in line and "Accepted password" not in line:
            continue

        parts = line.split()
        timestamp = " ".join(parts[0:2])
        host = parts[2]
        status = "FAIL" if "Failed" in line else "SUCCESS"

        user = None
        ip = None

        if "user" in parts:
            try:
                user_index = parts.index("user") + 1
                user = parts[user_index]
            except ValueError:
                pass

        ip = parts[-3]

        rows.append(
            {
                "timestamp": timestamp,
                "host": host,
                "username": user,
                "ip_address": ip,
                "status": status,
            }
        )

len(rows)


10

In [4]:
import pandas as pd

df = pd.DataFrame(rows)
df


,timestamp,host,username,ip_address,status
0,2025-12-02 09:01:12,host1,admin,192.168.1.10,FAIL
1,2025-12-02 09:01:45,host1,test,192.168.1.11,FAIL
2,2025-12-02 09:02:10,host1,None,192.168.1.50,SUCCESS
3,2025-12-02 09:03:02,host2,root,10.0.0.5,FAIL
4,2025-12-02 09:03:45,host2,admin,10.0.0.5,FAIL
5,2025-12-02 09:04:10,host2,admin,10.0.0.5,FAIL
6,2025-12-02 09:05:33,host1,None,192.168.1.50,SUCCESS
7,2025-12-02 09:06:01,host3,guest,172.16.0.20,FAIL
8,2025-12-02 09:06:30,host3,guest,172.16.0.20,FAIL
9,2025-12-02 09:07:10,host3,None,172.16.0.30,SUCCESS


In [5]:
failed_df = df[df["status"] == "FAIL"]

failed_by_ip = failed_df.groupby("ip_address")["status"].count().sort_values(ascending=False)
failed_by_user = failed_df.groupby("username")["status"].count().sort_values(ascending=False)

failed_by_ip, failed_by_user


(ip_address
 10.0.0.5        3
 172.16.0.20     2
 192.168.1.10    1
 192.168.1.11    1
 Name: status, dtype: int64,
 username
 admin    3
 guest    2
 root     1
 test     1
 Name: status, dtype: int64)

In [6]:
THRESHOLD = 3

suspicious_ips = failed_by_ip[failed_by_ip > THRESHOLD]
suspicious_ips


Series([], Name: status, dtype: int64)

In [7]:
df[df["ip_address"].isin(suspicious_ips.index)].sort_values("timestamp")


,timestamp,host,username,ip_address,status
